In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import teradata
import os
import re
import math

import fiona
from shapely.geometry import Point, shape 
import geopandas as gpd
import folium
import branca.colormap as cm

import configparser
import time
import requests
from urllib.parse import urlencode


In [ ]:
%%time
gps_df = pd.read_excel('franprix_gps.xlsx')
dept_geo = gpd.read_file('departements.geojson', driver='GeoJSON')
commune_geo = gpd.read_file('contours-geographiques-simplifies-des-communes-2019.geojson', driver='GeoJSON')

In [ ]:
commune_geo = commune_geo[['insee_dep', 'insee_com', 'nom_dep', 'nom_com', 'geometry']]
gps_df['CP'] = gps_df['CP'].apply(lambda x: ('00000%d' % x)[-5:])
gps_df['CODE_DEPT'] = gps_df['CP'].apply(lambda x: x[:2])

mags_geometry = [Point(xy) for xy in zip(gps_df.LNG, gps_df.LAT)]
crs = {'init': 'epsg:4326'}
gps_gdf = gpd.GeoDataFrame(gps_df, crs=crs, geometry=mags_geometry)

commune_geo['nom_dept_com'] = commune_geo.apply(lambda row: row['nom_dep'] + ':' + row['nom_com'], axis='columns')

In [ ]:
def fill_dist_dict(commune_polygon, store_point, nom_dept_com, dist_dict):
    dist = commune_polygon.boundary.distance(store_point)
    if not nom_dept_com in dist_dict:
        dist_dict[nom_dept_com] = dist
    else:
        if dist < dist_dict[nom_dept_com]:
            dist_dict[nom_dept_com] = dist
    return dist_dict


def calcul_shortest_distance(insee_dep, commune_polygon, nom_dept_com, store_dept, dist_dict, gps_gdf):
    if insee_dep in store_dept:
        dist_dict = gps_gdf[gps_gdf['CODE_DEPT'] == insee_dep].apply(lambda row:fill_dist_dict(commune_polygon,
                                                                                               row['geometry'],
                                                                                               nom_dept_com,
                                                                                               dist_dict),
                                                                     axis='columns')
    else:
        dist_dict = gps_gdf.apply(lambda row:fill_dist_dict(commune_polygon,
                                                            row['geometry'],
                                                            nom_dept_com,
                                                            dist_dict),
                                  axis='columns')

    return dist_dict

In [ ]:
%%time
store_dept = gps_df.CODE_DEPT.unique()
dist_dict = {}

dist_dict = commune_geo.apply(lambda row:calcul_shortest_distance(row['insee_dep'],
                                                                  row['geometry'],
                                                                  row['nom_dept_com'],
                                                                  store_dept,
                                                                  dist_dict,
                                                                  gps_df),
                              axis='columns')

commune_geo['shortest_distance'] = commune_geo['nom_dept_com'].map(dist_dict.iloc[0, 0])

In [ ]:
commune_visu_gdf = commune_geo.copy()
commune_visu_gdf = commune_visu_gdf[['nom_dept_com', 'geometry', 'shortest_distance']]
commune_visu_gdf['shortest_distance_km'] = commune_visu_gdf['shortest_distance'] * 100
commune_visu_gdf = gpd.GeoDataFrame(commune_visu_gdf, crs=crs, geometry=commune_visu_gdf['geometry'])

commune_visu_gdf['geometry_hull'] = commune_visu_gdf.geometry.convex_hull
commune_visu_gdf = commune_visu_gdf[['nom_dept_com', 'geometry_hull', 'shortest_distance_km']]
commune_visu_gdf = gpd.GeoDataFrame(commune_visu_gdf, crs=crs, geometry=commune_visu_gdf['geometry_hull'])
commune_visu_gdf = commune_visu_gdf.drop(columns='geometry_hull')

In [ ]:
colormap_commune = cm.StepColormap(
    colors=['#8e2d09', '#d45410', '#fe8e25',
            '#fed383', '#ffffce'],
    vmin=min(commune_visu_gdf['shortest_distance_km']),
    vmax=max(commune_visu_gdf['shortest_distance_km']),
    index=[0, 10, 50, 100, 200, round(commune_visu_gdf['shortest_distance_km'].max(), 0)])

In [ ]:
from branca.element import Template, MacroElement

template1 = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">

<body>

<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:22px; right: 20px; top: 20px;'>
     
<div class='legend-title'>Distances des communes<br>au Franprix le plus proche</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#8e2d09;opacity:1;'></span>0 - 10km</li>
    <li><span style='background:#d45410;opacity:1;'></span>10 - 50km</li>
    <li><span style='background:#fe8e25;opacity:1;'></span>50 - 100km</li>
    <li><span style='background:#fed383;opacity:1;'></span>100 - 200km</li>
    <li><span style='background:#ffffce;opacity:1;'></span>> 200km</li>

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 12px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 10px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 20px;
    width: 40px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

In [ ]:
template_title = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">

<body>

<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:0px; background-color:rgba(252, 255, 255, 0.8);
     border-radius:0px; padding: 10px; font-size:40px; color:rgb(252, 58, 2); left: 50px; top: 0px;'>
<div class='legend-title'>Distances des communes au Franprix le plus proche</div>
</div>

</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 12px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 10px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 20px;
    width: 40px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

In [ ]:
shops_map = folium.Map(location=[46.803354, 1.8883335], zoom_start = 7)
folium.TileLayer('cartodbpositron').add_to(shops_map)

style_function = lambda x: {
    'fillColor': colormap_commune(x['properties']['shortest_distance_km']),
    'color': '',
    'weight': 0,
    'fillOpacity': 1
}

folium.GeoJson(commune_visu_gdf,
               style_function = style_function,
               name='Commune').add_to(shops_map)

folium.GeoJson(dept_geo,
               style_function = lambda x: {
                   'color': 'gray',
                   'weight': 1,
                   'fillOpacity': 0},
               name='Departement').add_to(shops_map)

for i,r in gps_gdf.iterrows():
    folium.CircleMarker(location=[r.LAT, r.LNG],
                        radius=1, 
                        color='black',
                        fill_color='black',
                        fill=True).add_to(shops_map)

macro = MacroElement()
macro._template = Template(template1)
macro2 = MacroElement()
macro2._template = Template(template_title)

shops_map.get_root().add_child(macro)
shops_map.get_root().add_child(macro2)


In [ ]:
shops_map.save('franprix_map.html')